In [108]:
import pandas as pd
import numpy as np
import json, re, requests, urllib, time, os
from bs4 import BeautifulSoup
from data_collection import load_data
from data_collection import add_distance
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
hikes = load_data('wta-parks-data.json')

In [110]:
hikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3412 entries, 0 to 3411
Data columns (total 28 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   name                                                       3412 non-null   object 
 1   url                                                        3412 non-null   object 
 2   length                                                     1921 non-null   float64
 3   highest point                                              1492 non-null   float64
 4   gain                                                       1492 non-null   float64
 5   lat                                                        2273 non-null   float64
 6   lon                                                        2273 non-null   float64
 7   pass: Discover Pass                                        3412 non-null   int64  
 8   pass: Na

In [111]:
hikes.head(2)

,name,url,length,highest point,gain,lat,lon,pass: Discover Pass,pass: National Monument Fee,pass: National Park Pass,...,Fall foliage,Coast,Mountain views,Wildlife,Old growth,Summits,Ridges/passes,Established campsites,Waterfalls,Rivers
0,Luther Burbank Park,http://www.wta.org/go-hiking/hikes/luther-burb...,3.0,120.0,120.0,47.5890,-122.2270,0,0,0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Gem Lake,http://www.wta.org/go-hiking/hikes/gem-lake,10.0,4857.0,1800.0,47.4453,-121.4235,0,0,0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
dist, time = add_distance('wta-parks-data.csv')

In [ ]:
time, dist

In [112]:
test_url = hikes['url'][0]

In [129]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
chrome_options = Options()  
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(chrome_options=chrome_options)

soups = []
driver.get(test_url)
soup = BeautifulSoup(driver.page_source, 'html')
while soup.find('li', {'class':'next'}) == True:
    driver.find_element_by_link_text(soup.find('li', {'class':'next'}).find('a').text).click()
    soup = BeautifulSoup(driver.page_source, 'html')
    soups.append(soup)
driver.close()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"
          Next 5 items
          »
        "}
  (Session info: headless chrome=81.0.4044.138)


In [117]:
print(soup.prettify())

<html class="js flexbox flexboxlegacy canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers applicationcache svg inlinesvg smil svgclippaths csspositionsticky" lang="en" style="" xml:lang="en" xmlns:xlink="http://www.w3.org/1999/xlink">
 <head>
  <script src="https://m.addthis.com/live/red_lojson/300lo.json?si=5eba19c9ff62dc8c&amp;bkl=0&amp;bl=1&amp;pdt=360&amp;sid=5eba19c9ff62dc8c&amp;pub=ra-58bde71c818bb993&amp;rev=v8.28.5-wp&amp;ln=en&amp;pc=men&amp;cb=0&amp;ab=-&amp;dp=www.wta.org&amp;fp=go-hiking%2Fhikes%2Fluther-burbank-park&amp;fr=&amp;of=0&amp;pd=0&amp;irt=0&amp;vcl=0&amp;md=0&amp;ct=1&amp;tct=0&amp;abt=0&amp;cdn=0&amp;pi=1&amp;rb=0&amp;gen=100&amp

In [128]:
soup.find('li', {'class':'next'}).find('a').text

'\n          Next 5 items\n          »\n        '

False